In [ ]:
import avapi
import avstack
import pandas
import pickle

import os
from tqdm import tqdm
import numpy as np
import shutil

## Define Perception/Tracking Models

In [ ]:
class _AgentModel():
    """Base class for agent fusion algorithms"""
    def __init__(self, perception, tracking):
        self.perception = perception
        self.tracking = tracking

    def __call__(self, *args, **kwargs):
        return self.ingest(*args, **kwargs)
        
    def ingest(self):
        raise NotImplementedError

# -----------------------------
# PERCEPTION ONLY
# -----------------------------

class EgoCameraPerception(_AgentModel):
    """Only performs ego-based perception"""
    def __init__(self, perception, *args, **kwargs):
        super().__init__(perception, None)

    def ingest(self, ego_image, agents_images):
        return self.perception['ego']['camera'](ego_image)


class AgentsCameraPerception(_AgentModel):
    """Only performs agent-based perception"""
    def __init__(self, perception, *args, **kwargs):
        super().__init__(perception, None)

    def ingest(self, ego_image, agents_images):
        return [self.perception['agent']['camera'](image) for image in agents_images]


class EgoLidarPerception(_AgentModel):
    """Only performs ego-based perception"""
    def __init__(self, perception, *args, **kwargs):
        super().__init__(perception, None)

    def ingest(self, ego_pc, agents_pcs):
        return self.perception['ego']['lidar'](ego_pc)


class AgentsLidarPerception(_AgentModel):
    """Only performs ego-based perception"""
    def __init__(self, perception, *args, **kwargs):
        super().__init__(perception, None)

    def ingest(self, ego_pc, agents_pcs):
        return [self.perception['agent']['lidar'](pc) for pc in agents_pcs]


# -----------------------------
# PERCEPTION AND TRACKING
# -----------------------------

class CameraPerceptionAndTracking(_AgentModel):
    """Only performs ego-based perception and tracking"""
    def __init__(self, perception, *args, **kwargs):
        tracking = avstack.modules.tracking.tracker2d.BasicBoxTracker2D()
        super().__init__(perception, tracking)

    def ingest(self, ego_image, agents_images):
        tracks_ego = self.tracking(self.perception['ego']['camera'](ego_image))
        return tracks_ego

        
class LidarPerceptionAndTracking(_AgentModel):
    """Only performs ego-based perception and tracking"""
    def __init__(self, perception, *args, **kwargs):
        tracking = avstack.modules.tracking.tracker3d.BasicBoxTracker3D()
        super().__init__(perception, tracking)

    def ingest(self, ego_pc, agents_pcs):
        tracks_ego = self.tracking(self.perception['ego']['lidar'](ego_pc))
        return tracks_ego


# -----------------------------
# PERCEPTION AND TRACKING WITH FUSION
# -----------------------------

class FusionAtTrackingWithDetections(_AgentModel):
    """Treat detections from other agents as detections""" 
    def __init__(self, perception, n_agents, *args, **kwargs):
        tracking = avstack.modules.tracking.tracker3d.BasicBoxTracker3D()
        super().__init__(perception, tracking)

    def ingest(self, ego_pc, agents_pcs):
        dets_ego = self.perception['ego']['lidar'](ego_pc)
        dets_all = [self.perception['agents']['lidar'](agents_pcs[i]) for i in range(len(agents_pcs))]
        dets_all.append(dets_ego)
        for dets in random.shuffle(dets_all):  # shuffle to avoid bias
            tracks_out = self.tracking(dets)  # TODO: need to adjust the configuration of the tracker?
        return tracks_out


class FusionAtTrackingWithTracks(_AgentModel):
    """Treat tracks from other agents as detections""" 
    def __init__(self, perception, n_agents, *args, **kwargs):
        tracking = {
            'ego':avstack.modules.tracking.tracker3d.BasicBoxTracker3D(),
            'agents':[avstack.modules.tracking.tracker3d.BasicBoxTracker3D() for _ in range(n_agents)]
        }
        super().__init__(perception, tracking)

    def ingest(self, ego_pc, agents_pcs):
        dets_ego = self.perception['ego'](ego_pc)
        pseudo_dets_all = [self.tracking['agents'][i](self.perception['agents'](agents_pcs[i])) for i in range(len(agents_pcs))]
        pseudo_dets_all.append(dets_ego)
        for pseudo_dets in random.shuffle(pseudo_dets_all):  # shuffle to avoid bias
            tracks_out = self.tracking(pseudo_dets)  # TODO: need to adjust the configuration of the tracker?
        return tracks_out


class DedicatedFusionLiDAR(_AgentModel):
    """Performs distributed data fusion on tracks from other agents"""
    def __init__(self, perception, n_agents, *args, **kwargs):
        tracking = {
            'ego':avstack.modules.tracking.tracker3d.BasicBoxTracker3D(),
            'agents':[avstack.modules.tracking.tracker3d.BasicBoxTracker3D() for _ in range(n_agents)]
        }
        super().__init__(perception, tracking)

    def fusion(self, tracks_ego, tracks_agents):
        pass
    
    def ingest(self, ego_pc, agents_pcs):
        tracks_ego = self.tracking['ego'](self.perception['ego'](ego_pc))
        tracks_agents = [self.tracking['agents'][i](self.perception['agents'](agents_pcs[i])) for i in range(len(agents_pcs))]
        tracks_out = self.fusion(tracks_ego, tracks_agents)

In [ ]:
perception = {
    'ego':{
        'camera':avstack.modules.perception.object2dfv.MMDetObjectDetector2D(model='fasterrcnn', dataset='carla', epoch='latest', threshold=0.5),
        'lidar':avstack.modules.perception.object3d.MMDetObjectDetector3D(model='pointpillars', dataset='carla', epoch='latest', threshold=0.2)
    },
    'agent':{
        'camera':avstack.modules.perception.object2dfv.MMDetObjectDetector2D(model='fasterrcnn', dataset='carla', threshold=0.5),
        'lidar':avstack.modules.perception.object3d.MMDetObjectDetector3D(model='pointpillars', dataset='carla', threshold=0.2)
    }
}

perception_reversed = {
    'ego':{
        'camera':perception['agent']['camera'],
        'lidar':perception['agent']['lidar']
    },
    'agent':{
        'camera':perception['ego']['camera'],
        'lidar':perception['agent']['lidar']
    }
}

In [ ]:
# string is: algorithm-train_data-test_data
agents = [
    ('EgoCameraPerception-ego-ego',        EgoCameraPerception(perception),             'ego',   'ego'),
    ('AgentsCameraPerception-agent-agent', AgentsCameraPerception(perception),          'agent', 'agent'),
    ('AgentsCameraPerception-ego-agent',   AgentsCameraPerception(perception_reversed), 'ego',   'agent'),
    ('EgoLidarPerception-ego-ego',         EgoLidarPerception(perception),              'ego',   'ego'),
    ('AgentsLidarPerception-agent-agent',  AgentsLidarPerception(perception),           'agent', 'agent'),
    ('AgentsLidarPerception-ego-agent',    AgentsLidarPerception(perception_reversed),  'ego',   'agent')
]

## Test Perception Application

In [ ]:
data_dir_ego = '/data/spencer/CARLA/ego-lidar/'
data_dir_infra = '/data/spencer/CARLA/multi-agent-v1/'

SM_ego = avapi.carla.CarlaScenesManager(data_dir_ego)
SM_infra = avapi.carla.CarlaScenesManager(data_dir_infra)

In [ ]:
out_dir = 'application_results'
n_scenes = 2
n_frames = 30

# perform tests for each model
results = {}
print('Testing {} models'.format(len(agents)))
for i_model, (model_str, model, train, test) in enumerate(agents):
    results[model_str] = []
    
    # -- get test data loader
    if test == 'ego':
        SM = SM_ego
        vehicle = 'ego'
    elif test == 'agent':
        SM = SM_infra
        vehicle = 'agent'
    else:
        raise NotImplementedError(test)

    # -- determine data input type
    if 'camera' in model_str.lower():
        data_type = 'camera'
    else:
        data_type = 'lidar'

    # -- make save folders
    out_folder = os.path.join(out_dir, model_str)
    gt_dir = os.path.join(out_folder, 'ground-truth')
    det_dir = os.path.join(out_folder, 'detection-results')
    if os.path.exists(gt_dir):
        shutil.rmtree(gt_dir)
    if os.path.exists(det_dir):
        shutil.rmtree(det_dir)
    os.makedirs(gt_dir, exist_ok=False)
    os.makedirs(det_dir, exist_ok=False)
        
    # -- execute the model over the scene data
    n_scenes_run = min(n_scenes, len(SM.splits_scenes['val']))
    print('Running over {} scenes for model #{}/{} - {}'.format(n_scenes_run, i_model+1, len(agents), model_str))
    for i_scene, scene in enumerate(tqdm(SM.splits_scenes['val'][:n_scenes_run], position=0, leave=True)):
        SD = SM.get_scene_dataset_by_name(scene)

        # -- get all the frames for applicable sensors
        if data_type == 'camera':
            if vehicle == 'ego':
                sensors = [sensor for sensor in SD.sensor_IDs if
                               ('cam' in sensor.lower()) and
                               ('depth' not in sensor.lower()) and 
                               ('semseg' not in sensor.lower()) and 
                               ('infra' not in sensor.lower())]
            else:
                sensors = [sensor for sensor in SD.sensor_IDs if 
                               ('cam' in sensor.lower()) and
                               ('depth' not in sensor.lower()) and 
                               ('semseg' not in sensor.lower()) and 
                               ('infra' in sensor.lower())]
        else:
            if vehicle == 'ego':
                sensors = [sensor for sensor in SD.sensor_IDs if
                               ('lidar' in sensor.lower()) and
                               ('depth' not in sensor.lower()) and 
                               ('semseg' not in sensor.lower()) and 
                               ('infra' not in sensor.lower())]
            else:
                sensors = [sensor for sensor in SD.sensor_IDs if 
                               ('lidar' in sensor.lower()) and
                               ('depth' not in sensor.lower()) and 
                               ('semseg' not in sensor.lower()) and 
                               ('infra' in sensor.lower())]
                
        # -- loop sensors that apply
        for sensor in sensors:
            frames = SD.get_frames(sensor=sensor)
            # -- loop frames
            nframes_min = min(n_frames, len(frames))
            for frame in frames[:nframes_min]:                
                # -- get perception data
                try:
                    if data_type == 'camera':
                        data = SD.get_image(frame=frame, sensor=sensor)
                    else:
                        data = SD.get_lidar(frame=frame, sensor=sensor)
                    objs = SD.get_objects(frame=frame, sensor=sensor)
                except (FileNotFoundError, KeyError):
                    continue

                # -- determine if ego or agent data
                if vehicle == 'ego':
                    ego_data = data
                    agents_data = []
                else:
                    ego_data = None
                    agents_data = [data]

                # -- run perception model
                if data_type == 'camera':
                    outputs = model(ego_image=ego_data, agents_images=agents_data)
                else:
                    outputs = model(ego_pc=ego_data, agents_pcs=agents_data)

                # -- HACK: if it's an agent model, we are just considering one sensor at a time
                if vehicle != 'ego':
                    outputs = outputs[0]
                    
                # -- aggregate results
                truths = avstack.datastructs.DataContainer(
                    frame=frame,
                    timestamp=0.0,
                    data=[obj.box.project_to_2d_bbox(data.calibration) if data_type == 'camera' else obj.box for obj in objs],
                    source_identifier='truths',
                )
                res = avapi.evaluation.ResultManager(
                    idx=frame,
                    detections=outputs,
                    truths=truths,
                    metric='2D_IoU' if data_type == 'camera' else '3D_IoU',
                    threshold=0.5 if data_type == 'camera' else 0.3,
                    assign_algorithm='greedy',
                )
                results[model_str].append(res)

# save last results since it takes long AF...
out_file = 'perception_application_results.p'
with open(out_file, 'wb') as f:
    pickle.dump(results, f)
print('Just saved resutls to {}'.format(out_file))

## Analyze Results

In [ ]:
# load results
if 'results' not in locals():
    out_file = 'perception_application_results.p'
    with open(out_file, 'rb') as f:
        results = pickle.load(f)
    print('Just loaded resutls from {}'.format(out_file))
else:
    print('Results variable seems to already be defined...keep going!')

In [ ]:
# Separate into two for camera and lidar
ap_wrapped_by_sensor = {'camera':[], 'lidar':[]}
for model_str in results:
    ap, _, _ = avapi.evaluation.metrics.get_ap_from_results(results[model_str], by_class=True)    
    if 'camera' in model_str.lower():    ap_wrapped_by_sensor['camera'].append((model_str, ap))
    else:
        ap_wrapped_by_sensor['lidar'].append((model_str, ap))

# Make plots
for sensor, ap_wrapped in ap_wrapped_by_sensor.items():
    print('For sensor {}'.format(sensor))
    avapi.evaluation.metrics.barplot_ap(ap_wrapped)


## Visualize Results

In [ ]:
df = pandas.DataFrame()

In [ ]:
subrow_formatter = '\tworowsubtablecenter{{{}}}{{\tworowsubtablecenter{{{}}}{{{}}}}}'